<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/Fix/4_Testing_Fine_Tuned_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv('/content/test_df.csv')

In [3]:
test_df.head()

,Unnamed: 0,original_text,abstractive_summary,extractive_summary,original_text_length,abstractive_summary_length,extractive_summary_length
0,10293,liputan6 com jakarta ratusan mahasiswa se jabo...,ratusan mahasiswa se jabotabek memperingati ti...,liputan6 com jakarta ratusan mahasiswa se jabo...,108,20,17
1,4852,liputan6 com palembang puluhan orang perwakila...,puluhan warga muara enim sumsel mendatangi kan...,mereka menuntut perusahaan mengembalikan tanah...,189,29,51
2,9414,liputan6 com jakarta nangroe aceh darussalam d...,konflik di aceh dan maluku masih berlangsung s...,liputan6 com jakarta nangroe aceh darussalam d...,163,20,18
3,2291,liputan6 com jakarta tidak ada perbedaan peraw...,kepala pusat perencanaan dan pengembangan pt k...,perbedaan hanya pada pelayanan kepada para pen...,157,30,34
4,3272,liputan6 com jakarta menteri luar negeri demis...,sejumlah duta besar negara negara arab meminta...,pernyataan itu disampaikan alwi seusai bertemu...,105,19,51


In [4]:
columns_to_delete = ['Unnamed: 0', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']
# Delete the specified columns from the DataFrame
test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

,original_text,abstractive_summary,extractive_summary
0,liputan6 com jakarta ratusan mahasiswa se jabo...,ratusan mahasiswa se jabotabek memperingati ti...,liputan6 com jakarta ratusan mahasiswa se jabo...
1,liputan6 com palembang puluhan orang perwakila...,puluhan warga muara enim sumsel mendatangi kan...,mereka menuntut perusahaan mengembalikan tanah...
2,liputan6 com jakarta nangroe aceh darussalam d...,konflik di aceh dan maluku masih berlangsung s...,liputan6 com jakarta nangroe aceh darussalam d...
3,liputan6 com jakarta tidak ada perbedaan peraw...,kepala pusat perencanaan dan pengembangan pt k...,perbedaan hanya pada pelayanan kepada para pen...
4,liputan6 com jakarta menteri luar negeri demis...,sejumlah duta besar negara negara arab meminta...,pernyataan itu disampaikan alwi seusai bertemu...


# 1. Model : **kairaamilanii/BERT2BERT-ExtractiveSummarizer**

In [5]:
!pip install datasets transformers torch accelerate

In [6]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [ ]:
test_df = test_df.head()

In [7]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_extractive_summaries_for_df(df):
    model_path = "kairaamilanii/BERT2BERT-ExtractiveSummarizer"
    summarizer = EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token


    # Lists to store both types of summaries
    extractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            extractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

# Assuming test_df_subset is already defined
extractive_summary_BERT_df = generate_extractive_summaries_for_df(test_df)

extractive_summary_BERT_df

config.json:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Generating Summaries: 100%|██████████| 5/5 [00:07<00:00,  1.41s/it]


,extractive_summary
0,dalam aksinya mereka mengaku kecewa sebab hing...
1,mereka menuntut perusahaan mengembalikan tanah...
2,liputan6 com jakarta nangroe aceh darussalam d...
3,liputan6 com jakarta tidak ada perbedaan peraw...
4,menurut alwi sejak dulu pemerintah konsisten m...


In [ ]:
from google.colab import files

csv_file_path = 'extractive_summary_kaira_BERT_df.csv'
abstractive_summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

In [ ]:
# GENERATE BASED ON RAW MODEL
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_summaries_for_df(df):
    model_path = "cahya/bert2bert-indonesian-summarization"
    summarizer = EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token


    # Lists to store both types of summaries
    summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            summaries.append(summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'summary': summaries
    })

# Assuming test_df_subset is already defined
summary_BERT_df = generate_summaries_for_df(test_df)
summary_BERT_df

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating Summaries: 100%|██████████| 2991/2991 [37:41<00:00,  1.32it/s]


,summary
0,ratusan mahasiswa se jabotabek memperingati ti...
1,puluhan warga desa kota baru dan desa pagar gu...
2,ksad jenderal tni endriartono sutarto berharap...
3,tidak ada perbedaan perawatan antara kereta ap...
4,menlu demisioner alwi shihab menegaskan posisi...
...,...
2986,dpr dan pemerintah pusat didesak untuk segera ...
2987,buku kumpulan lukisan karya almarhum hendra gu...
2988,cerita percintaan antara sami un dan ing tay d...
2989,operasi ketupat lebaran yang bertujuan untuk m...


In [ ]:
from google.colab import files

csv_file_path = 'summary_BERT_raw.csv'
summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ROUGE calculation

In [8]:
!pip install datasets transformers torch accelerate rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6fe2fb6612215c5df48b733b24acaa79fbd85703b6fa8d6f541cbbcab08fcf89
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [9]:
extractive_summary_BERT_df = pd.read_csv('/content/extractive_summary_BERT_df.csv')
summary_BERT_df = pd.read_csv('/content/summary_BERT_raw.csv')

In [10]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
from transformers import pipeline

In [11]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_BERT_df['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"ROUGE fine-tuned BERT2BERT model extractive summary:")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

ROUGE fine-tuned BERT2BERT model extractive summary:
Average ROUGE-1: 0.48416238084327734
Average ROUGE-2: 0.399254182843347
Average ROUGE-L: 0.4478177775648456


In [12]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = summary_BERT_df['summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"ROUGE raw BERT2BERT model extractive summary:")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

ROUGE raw BERT2BERT model extractive summary:
Average ROUGE-1: 0.46801917505179136
Average ROUGE-2: 0.36160433473543696
Average ROUGE-L: 0.42796482420016035


# 2. Model : **kairaamilanii/BERT2BERT-AbstractiveSummarizer**

In [13]:
!pip install datasets transformers torch accelerate

In [14]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [15]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_abstractive_summaries_for_df(df):
    model_path = "kairaamilanii/BERT2BERT-AbstractiveSummarizer"
    summarizer = EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token


    # Lists to store both types of summaries
    abstractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            abstractive_summaries.append(abstractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            abstractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'abstractive_summary': abstractive_summaries
    })

# Assuming test_df_subset is already defined
abstractive_summary_BERT_df = generate_abstractive_summaries_for_df(test_df)

abstractive_summary_BERT_df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Generating Summaries: 100%|██████████| 2991/2991 [33:44<00:00,  1.48it/s]


,abstractive_summary
0,ratusan mahasiswa se jabotabek memperingati ti...
1,puluhan warga desa kota baru dan desa pagar gu...
2,ksad jenderal tni endriartono sutarto berharap...
3,kepala pusat perencanaan dan pengembangan pt k...
4,pemerintah menegaskan posisi pemerintah indone...
...,...
2986,dpr dan pemerintah pusat didesak segera membah...
2987,seluruh penjualan buku karya almarhum hendra g...
2988,sebuah sanggar lenong onteng digelar di situ b...
2989,operasi ketupat lebaran mulai digelar polda me...


In [ ]:
from google.colab import files

csv_file_path = 'abstractive_summary_BERT_df.csv'
abstractive_summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

BERTScore testing


In [16]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00


In [17]:
abstractive_summary_BERT_df = pd.read_csv('/content/abstractive_summary_BERT_df.csv')
summary_BERT_raw = pd.read_csv('/content/summary_BERT_raw.csv')

In [ ]:
abstractive_summary_BERT_df

In [19]:
import bert_score

references = test_df['abstractive_summary'].tolist()
candidates = abstractive_summary_BERT_df['abstractive_summary'].tolist()

P, R, F1 = bert_score.score(candidates, references, lang="id", model_type="bert-base-uncased")

# Aggregate scores (mean)
print(f"BERTScore fine-tuned BERT2BERT model abstractive summary:")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore fine-tuned BERT2BERT model abstractive summary:
BERTScore Precision: 0.7284
BERTScore Recall: 0.7395
BERTScore F1: 0.7336


In [20]:
import bert_score

references = test_df['abstractive_summary'].tolist()
candidates = summary_BERT_raw ['summary'].tolist()

P, R, F1 = bert_score.score(candidates, references, lang="id", model_type="bert-base-uncased")

# Aggregate scores (mean)
print(f"BERTScore raw BERT2BERT model abstractive summary:")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

BERTScore raw BERT2BERT model abstractive summary:
BERTScore Precision: 0.7144
BERTScore Recall: 0.7329
BERTScore F1: 0.7232


# 3. Model : **kairaamilanii/T5-ExtractiveSummarizer**

In [5]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00


In [6]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [7]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [8]:
#  generate summaries based on the model using the test_df

import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline
from tqdm import tqdm
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer

def generate_summaries_for_df(df):
    model_path = "cahya/t5-base-indonesian-summarization-cased"
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    summaries = []

    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            summary_ids = model.generate(**inputs, max_length=200)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")
            print(f"Error: {e}")
            summaries.append("")

    return pd.DataFrame({
        'summary': summaries
    })

summary_T5_raw = generate_summaries_for_df(test_df)
summary_T5_raw



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating Summaries: 100%|██████████| 2991/2991 [2:12:32<00:00,  2.66s/it]


,summary
0,Ratusan mahasiswa se jabotabek memperingati ti...
1,Puluhan warga desa kota baru dan desa pagar gu...
2,Kepala Staf tni angkatan darat jenderal tni en...
3,jakarta tidak ada perbedaan perawatan antara k...
4,Menteri Luar Negeri demisioner alwi shihab men...
...,...
2986,Tim otonomi khusus papua mendesak dpr dan peme...
2987,Liputan6 com jakarta buku kumpulan lukisan kar...
2988,Penyaluran cerita percintaan antara sami un da...
2989,kepolisian daerah metro jaya rabu 28 11 mulai ...


In [9]:
from google.colab import files

csv_file_path = 'summary_T5_raw.csv'
summary_T5_raw.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# generate summaries based on the model using the test_df

import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline
from tqdm import tqdm
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
from rouge_score import rouge_scorer

def generate_extractive_summaries_for_df(df):
    model_path = "kairaamilanii/T5-ExtractiveSummarizer"
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    extractive_summaries = []

    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            summary_ids = model.generate(**inputs, max_length=200)
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")
            print(f"Error: {e}")
            extractive_summaries.append("")

    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

extractive_summary_T5_df = generate_extractive_summaries_for_df(test_df)
extractive_summary_T5_df


In [ ]:
from google.colab import files

csv_file_path = 'extractive_summary_T5_df.csv'
extractive_summary_T5_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

ROUGE calculation

In [ ]:
!pip install datasets transformers torch accelerate rouge-score

In [ ]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
from transformers import pipeline

In [ ]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [21]:
extractive_summary_T5_df = pd.read_csv('/content/extractive_summary_T5_df.csv')
summary_T5_raw = pd.read_csv('/content/summary_T5_raw.csv')

In [23]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_T5_df['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"ROUGE fine-tuned T5 model extractive summary:")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

ROUGE fine-tuned T5 model extractive summary:
Average ROUGE-1: 0.5251607346805842
Average ROUGE-2: 0.4489396826470044
Average ROUGE-L: 0.4917028242319308


In [25]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = summary_T5_raw['summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"ROUGE raw T5 model extractive summary:")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

ROUGE raw T5 model extractive summary:
Average ROUGE-1: 0.41242164508185103
Average ROUGE-2: 0.3673287351930601
Average ROUGE-L: 0.3901658464137844


# 4. Model : **kairaamilanii/T5-AbstractiveSummarizer**

In [ ]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [ ]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [10]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline
from tqdm import tqdm
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
def generate_abstractive_summaries_for_df(df):
    model_path = "kairaamilanii/T5-AbstractiveSummarizer"
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    abstractive_summaries = []

    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            summary_ids = model.generate(**inputs, max_length=200)
            abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            abstractive_summaries.append(abstractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")
            print(f"Error: {e}")
            abstractive_summaries.append("")

    return pd.DataFrame({
        'abstractive_summary': abstractive_summaries
    })

abstractive_summary_T5_df = generate_abstractive_summaries_for_df(test_df)
abstractive_summary_T5_df


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Generating Summaries: 100%|██████████| 2991/2991 [36:09<00:00,  1.38it/s]


,abstractive_summary
0,ratusan mahasiswa se jabotabek memperingati ti...
1,puluhan warga desa kota baru dan desa pagar gu...
2,kepala staf tni afghan jenderal tni endriarton...
3,pt pt indonesia membantah tudingan diskriminas...
4,menteri luar negeri demisioner alwi shihab men...
...,...
2986,tim otonomi khusus papua mendesak dpr dan peme...
2987,buku kumpulan lukisan karya almarhum hendra gu...
2988,pembauran antara masyarakat tionghoa dan betaw...
2989,operasi ketupat lebaran yang digelar di stasiu...


In [11]:
from google.colab import files

csv_file_path = 'abstractive_summary_T5_df.csv'
abstractive_summary_T5_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BERTScore evaluation

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00


In [26]:
abstractive_summary_T5_df = pd.read_csv('/content/abstractive_summary_T5_df.csv')
summary_T5_raw = pd.read_csv('/content/summary_T5_raw.csv')

In [29]:
import bert_score

references = test_df['abstractive_summary'].tolist()
candidates = abstractive_summary_T5_df['abstractive_summary'].tolist()

P, R, F1 = bert_score.score(candidates, references, lang="id", model_type="bert-base-uncased")

# Aggregate scores (mean)
print(f"BERTScore fine-tuned T5 model abstractive summary:")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERTScore fine-tuned T5 model abstractive summary:
BERTScore Precision: 0.7277
BERTScore Recall: 0.7417
BERTScore F1: 0.7341


In [30]:
import bert_score

references = test_df['abstractive_summary'].tolist()
candidates = summary_T5_raw['summary'].tolist()

P, R, F1 = bert_score.score(candidates, references, lang="id", model_type="bert-base-uncased")

# Aggregate scores (mean)
print(f"BERTScore raw T5 model abstractive summary:")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

BERTScore raw T5 model abstractive summary:
BERTScore Precision: 0.6038
BERTScore Recall: 0.7631
BERTScore F1: 0.6727
